In [1]:
#Импорт необходимых библиотек

import requests
from bs4 import BeautifulSoup as bs
import sqlite3

In [2]:
#Парсинг страницы

url = 'https://quotes.toscrape.com/'
all_qoutes = []
p = 1

while True:
    print(f'Collecting data from page #{p}')
    req = requests.get(url)
    if req.status_code == 200:
        page = bs(req.content, 'html.parser')
        qoutes = page.find_all('div', class_='quote')
        
        for q in qoutes:
            author = q.find('small', {'class': 'author'}).text
            text = q.find('span', attrs={'class':'text'}).text
            tags = q.find('div', {'class':'tags'}).find('meta')['content']
    
            quotes_tuple = (author, text, tags.replace(',', ', '))
    
            all_qoutes.append(quotes_tuple)
        
        try:
            next_page = page.select_one('nav li[class=next] a')['href']
            url = f'https://quotes.toscrape.com/{next_page}'
            p += 1
        except:
            break

print('Done!')

Done!


In [3]:
#Создание базы данных Quotes

connection = sqlite3.connect('qoutes.db')
cursor = connection.cursor()

#Проверка и удаление существующей таблицы

delete = '''
    DROP TABLE IF EXISTS texts
'''
cursor.execute(delete)

#Создание новой таблицы

query = '''
    CREATE TABLE IF NOT EXISTS texts (
        author TEXT,
        quote TEXT,
        tags TEXT
    )
'''

cursor.execute(query)
connection.commit()

#Вставка значений в базу данных

insert_query = '''
    INSERT INTO texts (
        author,
        quote,
        tags
    )
    VALUES (
        ?,
        ?,
        ?
    )
'''

cursor.executemany(insert_query, all_qoutes)
connection.commit()

#Проверка

search_query = 'SELECT * FROM texts'
cursor.execute(search_query)
list(cursor.execute(search_query))[0]

('Albert Einstein',
 '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 'change, deep-thoughts, thinking, world')